In [4]:
!pip install numpy-financial

In [5]:
import numpy as np
import pandas as pd
from scipy import stats
import random as rd
import numpy_financial as npf

# 貸款參數
Loan_amount = 1500000  # 貸款金額
Loan_period = 36       # 貸款期數/月
Loan_rate = 3          # 貸款年利息 %

# 投資參數
Invest_Amount = 10000          # 每月投入金額
Invest_rate = 5                # 投資年報酬率 %
Invest_period = 3              # 幾個月配息一次
Expect_Price_Low = 15          # 投資商品預期價格低值
Expect_Price_High = 25         # 投資商品預期價格高值

# 初始化計算
Expect_Price_Ave = (Expect_Price_High + Expect_Price_Low) / 2
monthly_pay = npf.pmt(Loan_rate / 12 / 100, Loan_period, Loan_amount).round(2)

# 模擬次數
num_simulations = 1000
simulation_results = []

for sample in range(1, num_simulations + 1):
    # 初始條件
    initial_price = round(rd.uniform(Expect_Price_Low, Expect_Price_High), 2)
    QTY_No_Loan = round(Invest_Amount / initial_price, 2)
    QTY_Loan = round((Loan_amount + Invest_Amount) / initial_price, 2)
    Acc_Pay_No_Loan = 0
    Acc_Pay_Loan = 0

    for period in range(2, Loan_period + 1):
        # 更新價格
        price = round(rd.uniform(Expect_Price_Low, Expect_Price_High), 2)

        if period % Invest_period == 0:  # 配息月份
            # 無貸款方案計算
            Pay_No_Loan = round(QTY_No_Loan * Invest_rate / Invest_period / 100, 2)
            QTY_No_Loan += round((Invest_Amount + Pay_No_Loan) / price, 2)
            Acc_Pay_No_Loan += Pay_No_Loan

            # 貸款方案計算
            Pay_Loan = round(QTY_Loan * Invest_rate / Invest_period / 100, 2)
            QTY_Loan += round((Invest_Amount + monthly_pay + Pay_Loan) / price, 2)
            Acc_Pay_Loan += Pay_Loan
        else:  # 非配息月份
            QTY_No_Loan += round(Invest_Amount / price, 2)
            QTY_Loan += round((Invest_Amount + monthly_pay) / price, 2)

    # 記錄結果
    simulation_results.append({
        '模擬次數': f'第{sample}次',
        '未貸款期末單位數': round(QTY_No_Loan, 2),
        '貸款期末單位數': round(QTY_Loan, 2),
        '未貸款期末本金': round(QTY_No_Loan * Expect_Price_Ave, 2),
        '貸款期末本金': round(QTY_Loan * Expect_Price_Ave, 2),
        '未貸款累積配息': round(Acc_Pay_No_Loan, 2),
        '貸款累積配息': round(Acc_Pay_Loan, 2),
    })

# 將結果轉為 DataFrame
data = pd.DataFrame(simulation_results)

# 平均計算
ave_no_loan_amt = data['未貸款期末本金'].mean()
ave_loan_amt = data['貸款期末本金'].mean()
ave_no_loan_dvd = data['未貸款累積配息'].mean()
ave_loan_dvd = data['貸款累積配息'].mean()

# T檢定
statistic, p_value = stats.ttest_ind(
    data['未貸款期末本金'] + data['未貸款累積配息'],
    data['貸款期末本金'] + data['貸款累積配息']
)

# 結果輸出
if p_value < 0.05:
    print(f'貸款 {Loan_amount}, 期數 {Loan_period}, 每期還款金額 {monthly_pay}')
    print(f'模擬 {num_simulations} 次後，兩方案有明顯差異')
    print(f'經過 {Loan_period} 期後')
    if (ave_no_loan_amt + ave_no_loan_dvd) > (ave_loan_amt + ave_loan_dvd):
        print(f'平均未貸款方案可多賺 {round((ave_no_loan_amt - ave_loan_amt + ave_no_loan_dvd - ave_loan_dvd), 2)}')
    else:
        print(f'平均貸款方案可多賺 {round((ave_loan_amt - ave_no_loan_amt + ave_loan_dvd - ave_no_loan_dvd), 2)}')
else:
    print(f'貸款 {Loan_amount}, 期數 {Loan_period}, 每期還款金額 {monthly_pay}')
    print(f'模擬 {num_simulations} 次後，兩方案沒有明顯差異')
    print(f'經過 {Loan_period} 期後')
    print(f'未貸款方案平均收益為 {round(ave_no_loan_amt, 2)}')
    print(f'貸款方案平均收益為 {round(ave_loan_amt, 2)}')


貸款 1500000, 期數 36, 每期還款金額 -43621.81
模擬 1000 次後，兩方案沒有明顯差異
經過 36 期後
未貸款方案平均收益為 370070.98
貸款方案平均收益為 348006.59
